In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = .001

### Model:

In [ ]:
class FeedForwardNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_layers = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        flattened_data = self.flatten(input_data)
        logits = self.dense_layers(flattened_data)
        predictions = self.softmax(logits)
        return predictions

1. Download Dataset
   - Using 2 datasets: `train_data` will be the training data from PyTorch
     - `MNIST` is a datasetclass

In [ ]:
def download_mnist_datasets():
    train_data = datasets.MNIST(
        root="data",
        download=True,
        train=True,
        transform=ToTensor()
    )
    validation_data = datasets.MNIST(
        root="data",
        download=True,
        train=False,
        transform=ToTensor()
    )
    return train_data, validation_data

#### Train model:

In [ ]:
# Training Epoch of our Model:
def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
    # Loop through all the samples in dataset, in every iteration, we get a new batch of samples
    for inputs, targets in data_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Calculate the loss
        predictions = model(inputs)
        loss = loss_fn(predictions, targets)
        # backpropagate the loss and update the model weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print(f"Loss: {loss.item()}")
    

In [ ]:
# Higher Level Function which will call the train_one_epoch function for multiple epochs:
def train(model, data_loader, loss_fn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_one_epoch(model, data_loader, loss_fn, optimiser, device)
        print("---------------------------")
    print("Finished Training")

In [ ]:
if __name__ == "__main__":
    #Download MNIST data
    train_data, _ = download_mnist_datasets()
    print("Downloading MNIST Dataset Complete")
    
    # Create a data loader for the train_data
    train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
    
    # Check if GPU ACC is available
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print (f"Using the {device} device")   
    
    # Build Model
    feed_forward_net = FeedForwardNet().to(device)

    # instantiate loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(feed_forward_net.parameters(), lr=LEARNING_RATE)

    # Train Model:
    train(feed_forward_net, train_data_loader, loss_fn, optimiser, device, EPOCHS)

    # Save Model:
    torch.save(feed_forward_net.state_dict(), "trained-models/feedforwardnet.pth")
    print("Model trained and stored in feedforwardnet.pth")